In [7]:
import sys
sys.path.append('./python')

In [8]:
import needle as ndl
import needle.nn as nn
from needle import array_api, Tensor, NDArray
import numpy as np
from matplotlib import pyplot as plt

In [9]:
_device = "cuda"
model = nn.Unet()
optimizer = ndl.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
from pathlib import Path
from needle.autograd import Tensor


# def plot_landscape(tensor: Tensor):
#     img = tensor.permute((1, 2, 0)).numpy()
#     img = (img + 1) / 2
#     plt.imshow(img);


DATA_DIR = Path('./data/landscapes/')
IMAGE_FILES = sorted(str(x) for x in list(DATA_DIR.rglob('*.jpg')))

transforms = [
    ndl.data.RandomFlipHorizontal(),
    ndl.data.Lambda(lambda img: np.transpose(img, (2, 0, 1))),
]

dataset = ndl.data.LandscapesDataset(IMAGE_FILES, extra_transforms=transforms)
# plot_landscape(dataset[12])

In [11]:
timesteps = 300
module = nn.Diffusion(model, optimizer, timesteps, loss_type="l2")

In [12]:
epochs = 1
batch_size = 11
dataloader = ndl.data.DataLoader(dataset, batch_size)
device=ndl.cpu()

In [13]:
for epoch in range(epochs):
    for step, batch in enumerate(dataloader):
        # Algorithm 1 line 3: sample t uniformally for every example in the batch
        t = Tensor(np.random.randint(0, timesteps, (batch_size,)), device=device)

        loss = module.p_losses(batch, t)

        if step % 100 == 0:
            print("Loss:", loss.item())

        loss.backward()
        module.optimizer.step()

(11, 3, 256, 256) <-- x0 shape
(11, 1, 1, 1) <---- extract shape


ValueError: Wrong new_shape